<a href="https://colab.research.google.com/github/saotomryo/Use_MMDetection/blob/main/Use_MMDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import json
from skimage import measure
import numpy as np
print(torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device = ", device)

1.9.0+cu111
device =  cuda


## 環境準備 MMCV MMDetectionのインストール

In [2]:
# MMCVのインストール（30分程度かかります）

!pip install mmcv-full==1.3.8

     |████████████████████████████████| 312 kB 8.3 MB/s 
     |████████████████████████████████| 185 kB 45.0 MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.3.8-cp37-cp37m-linux_x86_64.whl size=29531064 sha256=6e5012e48150237bf67ae58f2e809c7dbd86717a567adc958afca57ef6c2c254
  Stored in directory: /root/.cache/pip/wheels/b1/c4/8d/b5bab7347e1b71280505e58eca95d00a53a524a7d5fa3ac89f
Successfully built mmcv-full


In [3]:
HOME_PATH = "/content"

In [4]:

!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -r requirements/build.txt
!pip install "git+https://github.com/open-mmlab/cocoapi.git#subdirectory=pycocotools"
!pip install -v -e .  # or "python setup.py develop"

Cloning into 'mmdetection'...
remote: Enumerating objects: 21477, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 21477 (delta 5), reused 5 (delta 0), pack-reused 21454
Receiving objects: 100% (21477/21477), 25.04 MiB | 25.92 MiB/s, done.
Resolving deltas: 100% (15062/15062), done.
/content/mmdetection
  Cloning https://github.com/open-mmlab/cocoapi.git to /tmp/pip-req-build-gftcm9_a
  Running command git clone -q https://github.com/open-mmlab/cocoapi.git /tmp/pip-req-build-gftcm9_a
  Created wheel for mmpycocotools: filename=mmpycocotools-12.0.3-cp37-cp37m-linux_x86_64.whl size=264251 sha256=8be12ea1f77a736ed4bf2f8c469af4dae2b85d507d37635d7127bec77005d5b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-n0ehthhf/wheels/d4/38/2b/ec4e44ec87a2b8692cd999bb33a831823b8071d0f89661297e
Successfully built mmpycocotools
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does n

## コンフィグファイルの編集

In [5]:
from mmcv import Config
cfg = Config.fromfile('/content/mmdetection/configs/yolo/yolov3_d53_mstrain-608_273e_coco.py')


In [7]:
%cd /content/

/content


In [15]:
# MMdetectionの分類結果は、COCODatasetで定義された80分類に従って、表示されます。
# 表示名を変更するには、以下のファイルの'cocodataset'の部分を編集してください。
# /content/mmdetection/mmdet/datasets/coco.py
# /content/mmdetection/mmdet/core/evaluation/class_names.py

# 編集後にすでに編集ずみファイルを持っている場合は以下のように内容をコピーして置き換えてください。


!cp "コピーもとファイル" "/content/mmdetection/mmdet/datasets/coco.py" -r
!cp "コピーもとファイル" "/content/mmdetection/mmdet/core/evaluation/class_names.py" -r 


In [13]:
# コンフィグを変更する
# yolov3

#data_root = '画像データのルートフォルダ'
data_root = '/content/'

cfg.model.bbox_head.num_classes = '分類するクラス数'
#cfg.num_class = 80
cfg.data_root = data_root

# trainデータの定義
cfg.data.train.ann_file = '学習用のアノテーションファイル'
cfg.data.train.img_prefix = data_root + '学習用画像データを保存したフォルダ'

# valデータの定義
cfg.data.val.ann_file = '検証用のアノテーションファイル'
cfg.data.val.img_prefix = data_root + '検証用画像データを保存したフォルダ'


# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


# 編集したコンフィグをファイル出力
cfg.dump('/content/yolo.py')


Config:
checkpoint_config = dict(interval=1)
log_config = dict(interval=50, hooks=[dict(type='TextLoggerHook')])
custom_hooks = [dict(type='NumClassCheckHook')]
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = None
workflow = [('train', 1)]
model = dict(
    type='YOLOV3',
    backbone=dict(
        type='Darknet',
        depth=53,
        out_indices=(3, 4, 5),
        init_cfg=dict(type='Pretrained', checkpoint='open-mmlab://darknet53')),
    neck=dict(
        type='YOLOV3Neck',
        num_scales=3,
        in_channels=[1024, 512, 256],
        out_channels=[512, 256, 128]),
    bbox_head=dict(
        type='YOLOV3Head',
        num_classes=4,
        in_channels=[512, 256, 128],
        out_channels=[1024, 512, 256],
        anchor_generator=dict(
            type='YOLOAnchorGenerator',
            base_sizes=[[(116, 90), (156, 198), (373, 326)],
                        [(30, 61), (62, 45), (59, 119)],
                        [(10, 13), (16, 30

In [16]:
!mkdir -p "$HOME_PATH"/work_train
!python /content/mmdetection/tools/train.py \
    '/content/yolo.py' \
    --work-dir "$HOME_PATH"/work_train


fatal: not a git repository (or any of the parent directories): .git
2021-10-28 01:31:15,405 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.12 (default, Sep 10 2021, 00:21:48) [GCC 7.5.0]
CUDA available: True
GPU 0: Tesla P100-PCIE-16GB
CUDA_HOME: /usr/local/cuda
NVCC: Build cuda_11.1.TC455_06.29190527_0
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.9.0+cu111
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.1.2 (Git Hash 98be7e8afa711dc9b66c8ff3504129cb82013cdb)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.1
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compu

## 学習結果の確認

In [ ]:
from argparse import ArgumentParser
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
model = init_detector("/content/yolo.py", "学習済みモデル", device=device)
result = inference_detector(model, "予測結果を確認する画像のパス")
show_result_pyplot(model, "予測結果を確認する画像のパス", result, score_thr=0.4)

## 予測タスクの実施

In [ ]:
from argparse import ArgumentParser
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
import os

testimage_path　 = "テスト画像をおいたフォルダ"

imgaes = glob(testimage_path)

# Specify the path to model config and checkpoint file
config_file = '/content/drive/MyDrive/Detection/mmdetection/configs/htc/htc_without_semantic_r50_fpn_1x_coco.py'
checkpoint_file = '学習済みモデルのパスを記載する'

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file,device=device)



bbox_dict1 = {}

for image in images:
    
    if os.path.exists(image):

        result = inference_detector(model, image)
        
        i = 0
        result_segmentation = []

        for i in range(6):
            result_segmentation1 = []
            tmp_bbox = []
            if len(result[0][i]):
                for j in range(len(result[0][i])):
                    if len(result_segmentation1) == 0:
                        contours = measure.find_contours(np.array(result[1][i][j]), 0.5)
                        result_segmentation1.append([contours[0].tolist(),result[0][i][j][0:5].tolist()])
                        tmp_bbox.append(result[0][i][j][0:4].tolist())

                    else:
                        if check_bbox(result[0][i][j][0:4].tolist(),tmp_bbox):
                            contours = measure.find_contours(np.array(result[1][i][j]), 0.5)
                            result_segmentation1.append([contours[0].tolist(),result[0][i][j][0:5].tolist()])
                            tmp_bbox.append(result[0][i][j][0:4].tolist())                       


                category = category_map[i+1]
                bbox_dict = {
                    category:result_segmentation1
                }

                if len(result_segmentation1):
                    result_segmentation.append(bbox_dict)
        
        bbox_dict1[img] = result_segmentation


with open('結果を出力するパス', 'w') as f:
    json.dump(bbox_dict1, f, ensure_ascii=False)



/content/drive/My Drive/Detection/mmdetection/mmdet/datasets/utils.py:68: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  'data pipeline in your config file.', UserWarning)
